In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def generate_noisy_signal(datapoints_num, tf):

    t = np.linspace(0, tf, datapoints_num)
    y = np.sin(t) + np.random.normal(0, 0.1, datapoints_num)

    return y.reshape(-1, 1)

In [3]:
def generate_input_output_seqs(y, encoder_inputseq_len, decoder_inputseq_len, stride = 1, num_features = 1):

    L = y.shape[0]
    num_samples = (L - encoder_inputseq_len - decoder_inputseq_len) // stride + 1

    train_input_seq = np.zeros((num_samples, encoder_inputseq_len, num_features))
    train_output_seq = np.zeros((num_samples, decoder_inputseq_len, num_features))

    for ff in np.arange(num_features):
        for ii in np.arange(num_samples):
            
            start_x = ii * stride
            end_x = start_x + encoder_inputseq_len
            train_input_seq[ii, :, ff] = y[start_x:end_x, ff]

            start_y = stride * ii + encoder_inputseq_len
            end_y = start_y + decoder_inputseq_len
            train_output_seq[ii, :, ff] = y[start_y:end_y, ff]

    return train_input_seq, train_output_seq

In [4]:
encoder_inputseq_len = 5
decoder_inputseq_len = 2
testing_seq_len = 50

y = generate_noisy_signal(2000, 80 * np.pi)
y_train = y[:testing_seq_len]

In [5]:
train_input_seq, train_output_seq = generate_input_output_seqs(y_train, encoder_inputseq_len, decoder_inputseq_len)

In [8]:
class Encoder(torch.nn.Module):

    def __init__(self, input_size, hidden_size, num_layers, dropout):

        super(Encoder, self).__init__()

        self.lstm = torch.nn.LSTM(input_size = input_size, hidden_size=hidden_size, num_layers=num_layers, 
                                  batch_first = True)

    def forward(self, input_seq, hidden_state):

        output, hidden = self.lstm(input_seq, hidden_state)

        return output, hidden

In [9]:
class Decoder(torch.nn.Module):

    def __init__(self, input_size, hidden_size, output_size, num_layers):

        super(Decoder, self).__init__()

        self.lstm = torch.nn.LSTM(input_size = input_size, hidden_size=hidden_size, num_layers=num_layers, 
                                  batch_first = True)
        self.fc_decoder = torch.nn.Linear(hidden_size, output_size)

    def forward(self, input_seq, encoder_hidden_state):

        output, hidden = self.lstm(input_seq, encoder_hidden_state)
        output = self.fc_decoder(output)

        return output, hidden

In [10]:
class Encoder_Decoder(torch.nn.Module):

    def __init__(self, input_size, hidden_size, decoder_output_size, num_layers):

        super(Encoder_Decoder, self).__init__()

        self.encoder = Encoder(input_size, hidden_size, num_layers)
        self.decoder = Decoder(input_size, hidden_size, decoder_output_size, num_layers)

In [15]:
### Quiz question
class Encoder(torch.nn.Module):

    def __init__(self, input_size, hidden_size, num_layers):

        super(Encoder, self).__init__()

        self.gru = torch.nn.GRU(input_size = input_size, hidden_size=hidden_size, \
                                num_layers=num_layers, batch_first = True)

    def forward(self, input_seq, hidden_state):

        output, hidden = self.gru(input_seq, hidden_state)

        return output, hidden
    
class Decoder(torch.nn.Module):

    def __init__(self, input_size, hidden_size, output_size, num_layers):

        super(Decoder, self).__init__()

        self.gru = torch.nn.GRU(input_size = input_size, hidden_size=hidden_size, num_layers=num_layers, 
                                  batch_first = True)
        self.fc_decoder = torch.nn.Linear(hidden_size, output_size)

    def forward(self, input_seq, encoder_hidden_state):

        output, hidden = self.lstm(input_seq, encoder_hidden_state)
        output = self.fc_decoder(output)

        return output, hidden
    
class Encoder_Decoder(torch.nn.Module):

    def __init__(self, input_size, hidden_size, decoder_output_size, num_layers):

        super(Encoder_Decoder, self).__init__()

        self.encoder = Encoder(input_size, hidden_size, num_layers)
        self.decoder = Decoder(input_size, hidden_size, decoder_output_size, num_layers)


# parameters
input_size = 1
hidden_size = 64
decoder_output_size = 1
num_layers = 1

model = Encoder_Decoder(input_size, hidden_size, decoder_output_size, num_layers)